In [35]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from loguru import logger
import torchvision
import torchvision.transforms as transforms
import numpy as np
from models import *

ckpt_path = "./checkpoint/ckpt.pth"

## Prepare data and build model

In [73]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

print('==> Preparing data..')
transform_train = transforms.Compose([????????????{":;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;po0-"}

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=True, num_workers=2)

print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = RegNetX_200MF()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Set up optimizer
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate,
                      momentum=0.9, weight_decay=5e-4)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


## Train and test functions

In [24]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss = train_loss + loss.item()
        _, predicted = outputs.max(1)
        total = total + targets.size(0)
        correct = correct + predicted.eq(targets).sum().item()
        
        logger.info("Epoch {}:\tLoss: {:.2f} | Acc: {:.2f}".format(epoch, (train_loss / (batch_idx+1)), 100.*correct/total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            acc = 100.*correct/total
        
        if acc > best_acc:
            print('Saving..')
            state = {
                'net': net.state_dict(),
                'acc': acc,
                'epoch': epoch,
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
                
            torch.save( {'epoch': epoch,
                        'model_state_dict': net.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': loss}, ckpt_path)
            best_acc = acc

## Begin training process

In [77]:
for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)

2020-10-26 10:16:03.520 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.46 | Acc: 12.50
2020-10-26 10:16:03.612 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.41 | Acc: 14.06
2020-10-26 10:16:03.703 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.48 | Acc: 11.46
2020-10-26 10:16:03.791 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.46 | Acc: 12.70
2020-10-26 10:16:03.879 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.46 | Acc: 12.19
2020-10-26 10:16:03.968 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.44 | Acc: 13.02
2020-10-26 10:16:04.056 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.48 | Acc: 12.17
2020-10-26 10:16:04.145 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.49 | Acc: 12.11
2020-10-26 10:16:04.233 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.49 | Acc: 12.15
2020-10-26 10:16:04.322 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.49 | Acc: 12.50
2020-10-26 10:16:04.410 | INFO     | __main__:train:19 - Epoch 0:	Loss: 2.48 | Acc: 12.93
2020-10-26

KeyboardInterrupt: 

In [28]:
trained_model = torch.nn.DataParallel(RegNetX_200MF())
checkpoint = torch.load(ckpt_path)
trained_model.load_state_dict(checkpoint['model_state_dict'])
trained_model.eval()

DataParallel(
  (module): RegNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): Block(
        (conv1): Conv2d(64, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3, bias=False)
        (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(64, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [104]:
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=True, num_workers=2)

total = 0
correct = 0
cur_test_loss = 0
for idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = trained_model(inputs)
    cur_loss = criterion(outputs, targets)

    cur_test_loss += cur_loss.item()
    _, predicted = outputs.max(1)
    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()
    #acc = 100.*correct/total

logger.info("Regular test data:\tLoss: {:.2f} | Acc: {:.2f}".format(cur_test_loss/100, 100.*correct/total))


2020-10-26 10:46:02.180 | INFO     | __main__:<module>:18 - Regular test data:	Loss: 0.42 | Acc: 87.53


In [108]:
# Now that we have a benchmark accuracy for the test dataset, let's poke a hole in each test image and try again. 
testset = torchvision.datasets.CIFAR10(
    root='./data/bad_data', train=False, download=True)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=True, num_workers=2)

from PIL import Image
from IPython.display import display
mported_Img = Image.open('ImageName.jpg')



Files already downloaded and verified
